In [1]:
import pickle
#加载模型
with open('model/gnb.pickle', 'rb') as f:
    model = pickle.load(f)
import pandas as pd
from datetime import date
current_date = date.today()
filename = './dataset/test.xls' #这里输入需要预测的文件名，格式为xls

#处理数据
table=pd.read_excel(filename)
r=table.groupby('用户编号')['缴费日期'].max().reset_index()
r['R']=(pd.to_datetime(current_date) - r['缴费日期']).dt.days
r=r[['用户编号','R']]
#引入日期标签辅助列
table['日期标签'] = table['缴费日期'].astype(str).str[:10]
#把单个用户一天内订单合并 主要是将单日的付款日期设置为1，因为一个客户不可能在同一时刻下单多次
dup_f = table.groupby(['用户编号','日期标签'])['缴费日期'].count().reset_index()
#对合并后的用户统计频次
f = dup_f.groupby('用户编号')['缴费日期'].count().reset_index()
f.columns = ['用户编号','F']
sum_m = table.groupby('用户编号')['缴费金额（元）'].sum().reset_index()
sum_m.columns = ['用户编号','总支付金额']
com_m = pd.merge(sum_m,f,left_on = '用户编号',right_on = '用户编号',how = 'inner')
com_m['M'] = com_m['总支付金额']
com_m['R'] = r['R']
com_m=com_m[["用户编号",'R','F','M']]
mean_num1=com_m['R'].mean()
std_num1=com_m['R'].std()
mean_num2=com_m['F'].mean()
std_num2=com_m['F'].std()
mean_num3=com_m['M'].mean()
std_num3=com_m['M'].std()
def func1(val):
    num=(val-mean_num1)/std_num1
    return num
def func2(val):
    num=(val-mean_num2)/std_num2
    return num
def func3(val):
    num=(val-mean_num3)/std_num3
    return num
com_m.R=com_m.apply(lambda x: func1(x.R),axis=1)
com_m.F=com_m.apply(lambda x: func2(x.F),axis=1)
com_m.M=com_m.apply(lambda x: func3(x.M),axis=1)
com_m['CLV']=(-0.2)*com_m['R']+0.35*com_m['F']+0.45*com_m['M']
com_m['default']=0
com_m

,用户编号,R,F,M,CLV,default
0,1000000001,0.163883,2.014043,1.321820,1.266957,0
1,1000000002,0.138560,0.319689,-1.312121,-0.506275,0
2,1000000003,0.113237,0.319689,2.344658,1.144340,0
3,1000000004,-6.774613,2.014043,-0.617863,1.781799,0
4,1000000005,0.113237,0.319689,4.040555,1.907494,0
5,1000000006,0.113237,0.319689,1.443713,0.738915,0
6,1000000007,0.113237,0.319689,1.952482,0.967861,0
7,1000000008,0.113237,0.319689,1.782892,0.891545,0
8,1000000009,0.113237,0.319689,1.231725,0.643520,0
9,1000000010,0.087914,2.014043,0.187689,0.771792,0


In [2]:
r=com_m.sort_values(by='CLV',ascending=False)
num=r['CLV'].index
num=num[0]
num

4

In [3]:
X=com_m[['CLV','default']]
res=model.predict(X)
res=pd.DataFrame(res)
res

,0
0,3
1,0
2,3
3,3
4,3
5,6
6,6
7,6
8,6
9,6


In [4]:
ans=res[0].loc[num]+1#获取类别
ans=str(ans)

In [5]:
p_table=pd.DataFrame(model.predict_proba(X))
p_table.columns=['1','2','3','4','5','6','7']
result=p_table.sort_values(by=ans,ascending=False)
result

,1,2,3,4,5,6,7
4,5.820158e-63,2.007992e-46,5.192350e-31,1.000000e+00,0.000000e+00,1.789833e-38,1.423545e-13
3,3.196880e-56,6.183243e-40,9.438074e-30,1.000000e+00,0.000000e+00,3.706771e-35,7.110849e-11
0,4.468308e-32,6.243738e-18,3.041561e-24,9.870477e-01,0.000000e+00,1.359212e-22,1.295227e-02
2,3.063010e-27,7.210978e-14,6.072976e-23,7.894272e-01,0.000000e+00,6.163982e-20,2.105728e-01
6,1.649241e-21,1.985987e-09,1.085267e-21,1.206442e-01,1.757585e-291,5.957571e-17,8.793558e-01
7,2.652606e-19,7.603121e-08,3.372433e-21,4.559387e-02,3.349419e-276,8.647374e-16,9.544060e-01
15,1.159548e-16,5.060094e-06,1.785154e-20,1.754935e-02,6.435671e-258,2.600455e-14,9.824456e-01
9,6.260277e-16,1.572422e-05,3.037854e-20,1.420505e-02,7.148036e-253,6.990944e-14,9.857792e-01
5,5.126776e-15,6.347692e-05,6.164016e-20,1.130334e-02,1.339363e-246,2.469814e-13,9.886332e-01
8,2.145989e-12,3.103098e-03,6.235013e-19,7.335928e-03,1.018559e-228,1.110841e-11,9.895610e-01


In [6]:
ans=com_m.loc[result.head().index.tolist(),:]
ans

,用户编号,R,F,M,CLV,default
4,1000000005,0.113237,0.319689,4.040555,1.907494,0
3,1000000004,-6.774613,2.014043,-0.617863,1.781799,0
0,1000000001,0.163883,2.014043,1.321820,1.266957,0
2,1000000003,0.113237,0.319689,2.344658,1.144340,0
6,1000000007,0.113237,0.319689,1.952482,0.967861,0


In [7]:
#获取用户编号，展示以及持久化
df=pd.DataFrame(ans,columns=['用户编号'])
df.index=['1','2','3','4','5']
print("最有可能成为高价值客户的TOP5:\n")
df

最有可能成为高价值客户的TOP5:



,用户编号
1,1000000005
2,1000000004
3,1000000001
4,1000000003
5,1000000007


In [8]:
df.to_csv('居民客户的用电缴费习惯分析 3.csv')